# Desafío 03-B-Chunking

## 1. Descripción general

En este cuaderno, repasarás los conceptos de tokens y chunking. En el cuaderno anterior (CH-03-A-Grounding), pudimos proporcionar algo de contexto adicional para fundamentar el modelo. ¿Existe un límite para la cantidad de contexto adicional que podemos proporcionar al modelo? Desafortunadamente, la respuesta es sí. Existe un límite para la cantidad de tokens permitidos en la entrada y la salida combinadas según el modelo que se esté utilizando.

Entonces, ¿qué son los tokens? Los tokens son una representación de cómo los modelos de Azure OpenAI procesan el texto. Son palabras o simplemente fragmentos de caracteres. Veamos el número total de tokens en la respuesta que obtuvimos del primer cuaderno en CH-03. Hay muchas formas de calcular tokens. En este cuaderno, echaremos un vistazo a la biblioteca `token` para contar los tokens.

## 2. Empecemos la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en los desafíos anteriores.

In [ ]:
! pip install --upgrade click
! python -m spacy download en_core_web_sm

In [ ]:
import openai
import PyPDF3
import os
import json
import tiktoken
import spacy
from openai.error import InvalidRequestError

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from spacy.lang.en import English 
nlp = spacy.load("en_core_web_sm")

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto final y claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [ ]:
# Load your OpenAI credentials
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
model=os.getenv("CHAT_MODEL_NAME")


## 3. Conteo de Tokens

Tiktoken utiliza una técnica llamada codificación de pares de bytes (BPE) para convertir el texto dado en tokens. Hay diferentes codificaciones disponibles para ayudar a procesar las palabras. En este cuaderno, utilizaremos el cl100k_base.

#### Tarea del Estudiante #1:

Cuenta el número de tokens en la respuesta final que recibimos en CH-03-A-Grounding completando la función, count_tokens, a continuación.

In [ ]:

def count_tokens(string: str, encoding_name: str) -> int:
    # Enter code here
    return 

#### Tarea del Estudiante #2:

Ingresa el texto de la respuesta que recibiste en CH-03-A-Grounding. Ejecuta la celda a continuación para recuperar el número de tokens utilizando la función count_tokens.

In [ ]:
text = """Enter text here."""

count_tokens(text, "cl100k_base")

print("There are " + str(count_tokens(text, "cl100k_base")) + " tokens: " + text)

De acuerdo, ahora sabemos con cuántos tokens estamos trabajando. ¿Qué sucede si queremos agregar más contexto que el que ya pusimos en la variable de texto anterior? Si pensamos en nuestro escenario de Wimbledon, necesitaremos darle al modelo más contexto para ayudarlo a entender todo lo que necesita saber sobre el torneo. Más importante aún, todo lo que necesita saber para ayudar a responder tus preguntas al escribir el informe. Digamos que queremos proporcionar más contexto al modelo con un documento PDF. ¿Podemos intentar obtener un resumen del documento PDF para ayudarnos con nuestro trabajo?

#### Tarea del Estudiante #3:

En la celda siguiente, inserta la ruta del documento PDF, `CH3-data.pdf`, que se encuentra en la carpeta `/data` proporcionada. Ejecuta las tres celdas para ver la salida.

In [ ]:
document = open(r'Insert PDF file path', 'rb') 
doc_helper = PyPDF3.PdfFileReader(document)

In [ ]:
finaltext = ''
totalpages = doc_helper.getNumPages()
for eachpage in range(totalpages):
   p = doc_helper.getPage(eachpage)
   indpagetext = p.extractText()
   finaltext += indpagetext

clean_text = finaltext.replace("  ", " ").replace("\n", "; ").replace(';',' ')

In [ ]:
prompt = f"What is the answer to the following question regarding the PDF document?\n\n{finaltext}\n\n" 
q = "Can you give me a summary of the document?"

try:
    final_prompt = prompt + q
    messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": final_prompt}]
    response = openai.ChatCompletion.create(engine=model, messages=messages, max_tokens=50)
    answer = response.choices[0].text.strip()
    print(f"{q}\n{answer}\n")

except InvalidRequestError as e:
    print(e.error)

Como verás arriba, recibirás un mensaje de error después de ejecutar el fragmento de código anterior. El modelo alcanza su longitud de contexto máxima. Para los modelos GPT-3, el límite de tokens es de 4097 tokens. ¿Cómo solucionamos este problema al proporcionarle todo el contexto necesario, pero sin encontrarnos con el problema del límite de tokens?

Para resolver este problema, podemos echar un vistazo a un concepto llamado Chunking.

## 4. Chunking

El chunking ayuda a limitar la cantidad de información que pasamos al modelo. La información que pasaremos son los trozos más relevantes de los datos en general. Hay muchas consideraciones que entran en juego al hacer chunking. Por ejemplo, necesitas averiguar el mejor tamaño de chunk. Si los trozos son demasiado pequeños, puedes perder contexto importante. Si los trozos son demasiado grandes, pueden contener información innecesaria.

A continuación se presentan algunas técnicas comunes de chunking.

1. Chunking con trozos más pequeños
2. Chunking dividiendo las oraciones
3. Chunking con superposición de oraciones
4. Chunking de forma recursiva

Veamos estas técnicas en acción.

### 4.1 Chunking con trozos más pequeños

#### Tarea del Estudiante #4:

Agrega código en la celda siguiente. Utiliza la función split() para dividir el texto en trozos.

In [ ]:

text = "The sun was setting over the horizon, casting a warm glow over the landscape. Birds chirped in the trees, and a gentle breeze rustled the leaves. In the distance, a herd of deer grazed in a meadow. The air was filled with the sweet scent of blooming flowers. It was a peaceful and serene scene, perfect for a quiet evening stroll."

# Add you code here

¿Qué puedes observar sobre los trozos devueltos? ¿Si cada trozo estuviera por sí mismo, podrías entender el significado semántico?

### 4.2: Chunking dividiendo las oraciones

#### Tarea del Estudiante #5:

Agrega código en la celda siguiente. Utiliza la biblioteca spacy y específicamente la función sents para dividir el texto en trozos.

In [ ]:
text = "Today was a fun day. I had lots of ice cream. I also met my best friend Sally and we played together at the new playground."

# Add you code here

¿Los resultados son mejores que el método en 4.1? La biblioteca spaCy ayuda a dividir el texto en oraciones individuales. Esto puede ser útil cuando intentas hacer resúmenes de texto. Puedes clasificar las oraciones individuales y usar los mejores resultados en el resumen.

### 4.3: Chunking con superposición de oraciones

#### Tarea del Estudiante #6:

Ejecuta el código a continuación para ver otro ejemplo de chunking. Como verás, el significado semántico se mantiene. En otras palabras, se conserva el contexto entre las oraciones. Esto es especialmente importante cuando estás buscando datos para obtener resultados relevantes o cuando estás resumiendo un fragmento de texto. Es importante capturar las relaciones entre las oraciones.

In [ ]:
text = "The sun was setting over the horizon, casting a warm glow over the landscape. Birds chirped in the trees, and a gentle breeze rustled the leaves. In the distance, a herd of deer grazed in a meadow. The air was filled with the sweet scent of blooming flowers. It was a peaceful and serene scene, perfect for a quiet evening stroll."
doc = nlp(text)

sentences = list(doc.sents)
overlap = 1
chunks =[]

for i in range(len(sentences) - overlap):
    chunk = sentences[i : i + overlap + 1]
    chunks.append(chunk)

for chunk in chunks:
    print([sent.text for sent in chunk])

### 4.4: Chunking de forma recursiva usando LangChain

#### Tarea del Estudiante #7:

Agrega los parámetros requeridos para RecursiveCharacterSplitter en la celda siguiente.

In [ ]:
split_text = RecursiveCharacterTextSplitter(
   # Add parameters here 
)
docs = split_text.create_documents([clean_text])
docs

Arriba, realizamos algunos fragmentos utilizando Langchain, un marco popular para crear aplicaciones utilizando modelos de lenguaje grandes. En los métodos anteriores, viste varios ejemplos de chunking. Langchain puede ayudar a hacer que el proceso de chunking sea más fácil con algunos de sus métodos. Estos métodos incluyen fragmentos de tamaño fijo, así como fragmentación recursiva, que acabamos de ver.

Por ejemplo, existe CharacterTextSplitter que dividirá el texto dado en un fragmento de tamaño fijo de un tamaño dado y un solapamiento de caracteres dado.

RecursiveCharacterTextSplitter divide el texto en fragmentos más pequeños de manera iterativa. Nuevamente, puedes proporcionar el tamaño del fragmento y el recuento de solapamiento del fragmento.

El chunking es una técnica importante por muchas razones. Ayuda a evitar el límite de tokens cuando se trabaja con grandes cantidades de datos y también optimiza la respuesta que recibimos del modelo. Encontrar la técnica de fragmentación adecuada y el tamaño de fragmento es crucial para recibir respuestas relevantes.

## Criterios de Éxito

Para completar este desafío con éxito:

* Mostrar comprensión de los tokens y cómo calcularlos.
* Demostrar comprensión del chunking experimentando con diferentes técnicas.
* Ser capaz de entender la importancia de encontrar la solución de chunking adecuada según si se captura o no el significado semántico.